# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [ ]:
from utils.helpers import get_api_key
import os

os.environ['OPENAI_API_KEY'] = get_api_key()

In [ ]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores.docarray import DocArrayInMemorySearch
# from langchain.vectorstores import DocArrayInMemorySearch
from langchain.llms import OpenAI

from IPython.display import display, Markdown

In [ ]:
loader = CSVLoader('./data/OutdoorClothingCatalog_1000.csv')

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch, embedding=embeddings
).from_loaders([loader])

In [ ]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [ ]:
llm_replacement_model = OpenAI(temperature=0, 
                               model='gpt-3.5-turbo-instruct')

response = index.query(query, 
                       llm = llm_replacement_model)

In [ ]:
display(Markdown(response))

## Step By Step

In [ ]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path='./data/OutdoorClothingCatalog_1000.csv')

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

In [ ]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [ ]:
len(embed)

In [ ]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [ ]:
query = "Please suggest a shirt with sunblocking"

In [ ]:
sim_docs = db.similarity_search(query)

In [ ]:
len(docs), len(sim_docs)

In [ ]:
sim_docs

In [ ]:
sim_docs[0]

In [ ]:
retriever = db.as_retriever()

In [ ]:
llm = ChatOpenAI(temperature = 0.9, model=llm_model)

In [ ]:
qdocs = "".join([sim_docs[i].page_content for i in range(len(sim_docs))])

In [ ]:
qdocs

In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


display(Markdown(response))


In [ ]:
f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one."

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [ ]:
response = qa_stuff.run(query)

In [ ]:
display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
index

In [ ]:
display(Markdown(response))